<a href="https://colab.research.google.com/github/ellen008/DeepL/blob/master/Final__Differential_Privacy_Project_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
      #++++++++++++
      #+++++++++split MNIST train dataset on train.train/train.valid

      #+++++++++create 10 teachers' datasets from train.train //train_ds//
      #+++++++++split 10 teachers' train.train dataset in train/valid
      #output:
      #teacher_train_loader = [1 ... 10]
      #teacher_valid_loader = [1 ... 10]

      #++++++++++++++++++++++++++
      #+++++++++train_with_teacher() 10 models on 10 splitted trainsets

      #++++++++++++++++++++++++++++   
      #10 teachers model.forward(train.valid_dataset) //valid_ds//
      #return 10 sets of true_labels = []  and mnist_labels = []

      #+++++++++++++++++
      #PATE analysis to figure out epsilon
      #dp_labels = true_labels + laplacianM(epsilon, beta)

      #++++++++++++++++++++++++++++++++
      #split train.valid_dataset and true_labels on train/valid
      #train.valid.train_loader
      #train.valid.valid_loader

      #+++++++++++++++++
      #train_local_model(train.valid_dataset + dp_labels)

      #++++++++++++++++++
      #predict(MNIST test_dataset)
      #compare predictions accuracy for true labels and dp_labels
    

#Download and transform data

In [0]:
import torch
import numpy as np
import torchvision.datasets as datasets
from torchvision import datasets, transforms
from torch.utils.data import random_split, SubsetRandomSampler, SequentialSampler
from torch import nn, optim
import torch.nn.functional as F

In [0]:
#download the data

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(),download = True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

#mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)


# Datasets (data + labels):

In [4]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train

In [5]:
test_dataset

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test

In [0]:
def split_dataset(dataset, ratio):
  
    train_size = int(ratio * len(dataset))
    valid_size = len(dataset) - train_size

    train_ds, valid_ds = torch.utils.data.random_split(dataset, [train_size, valid_size])
  
    return train_ds,valid_ds

In [7]:
#split train dataset into train.train = train_ds/ train.valid = valid_ds

test_train_ds, test_valid_ds = split_dataset(test_dataset,0.95)

print(f"test_train dataset = {len(test_train_ds)}",
      f"\ntest_valid dataset= {len(test_valid_ds)}")
print()


test_train dataset = 9500 
test_valid dataset= 500



# Create model

In [0]:
class Conv(nn.Module):
    
    def __init__(self):
        super(Conv, self).__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size = 5)
        self.conv2 = nn.Conv2d(10,20, kernel_size = 5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self,x):
        
        x = F.max_pool2d(self.conv1(x),2)
        x = F.relu(x)
        x = F.max_pool2d(self.conv2_drop(self.conv2(x)),2)
        x = F.relu(x)
        x = x.view(-1,320)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)
        x = F.log_softmax(x,dim = 1)
        
        return x
      

In [9]:
model = Conv()
#local_model = Conv()
model

Conv(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [0]:
#reset the model
def reset_model_conv(model):
    model.conv1.weight.data.uniform_(0.0, 1.0)
    model.conv1.bias.data.fill_(0)
    model.conv2.weight.data.uniform_(0.0, 1.0)
    model.conv2.bias.data.fill_(0)
    model.fc1.weight.data.uniform_(0.0, 1.0)
    model.fc1.bias.data.fill_(0)
    model.fc2.weight.data.uniform_(0.0, 1.0)
    model.fc2.bias.data.fill_(0)
    return -1

In [0]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [0]:
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

# train model_1

In [0]:
#loaders for training a single model on
#train.valid = valid_ds


#trainloader = torch.utils.data.DataLoader(dataset=train_ds, batch_size=64,shuffle = True)
#testloader = torch.utils.data.DataLoader(dataset=valid_ds, batch_size=64,shuffle = True)

trainloader = torch.utils.data.DataLoader(dataset = p_train_ds,  batch_size=64,shuffle = True)
testloader = torch.utils.data.DataLoader(dataset = p_valid_ds, batch_size=64,shuffle = True)

In [0]:
#reset conv network:
reset_model_conv()

-1

In [0]:
def evaluate_model_1(device,criterion,optimizer):
    
    #device = device
    test_loss = 0
    acc = 0

    model.eval()


    with torch.no_grad():
                
         for inputs, labels in testloader:

            #inputs = inputs.view(inputs.shape[0], -1)
            inputs, labels = inputs.to(device), labels.to(device)

            log_ps = model.forward(inputs)
            
            #print(labels.shape,log_ps.shape)
            test_loss += criterion(log_ps, labels).item()

            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            
            #print(f"top_p: {top_p}")
            equals = top_class == labels.view(*top_class.shape)
            #print(f"equals: {equals}")
            acc += torch.mean(equals.type(torch.FloatTensor))
            #print(f"acc: {acc}")
            #test_count +=1
            
    return test_loss, acc      


In [0]:
def train_model_1():
    
    device = 'cuda'
    epochs = 10
    count = 0

    model.to(device)

    criterion = nn.NLLLoss()
    #optimizer = optim.Adam(model.parameters(),lr = 0.01)
    optimizer = optim.SGD(model.parameters(),lr= 0.01, momentum = 0.5)
    
    train_losses = []
    test_losses = []
    accuracy =  []
    
    for e in range(epochs):
        model.train()
        running_loss = 0
        train_count = 0

        for images, labels in trainloader:
            # Flatten MNIST images into a 784 long vector
            #images = images.view(images.shape[0], -1)

            images, labels = images.to(device), labels.to(device)

            # TODO: Training pass
            optimizer.zero_grad()

            #output = model(images)
            output = model.forward(images)

            #print(output.shape, labels.shape)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            #print(train_count)
            train_count+=1
            
        else:
            #model evaluation
            test_loss,acc = evaluate_model_1(device,criterion,optimizer)

            #collect accuracy, train_losses and test_losses for each epoch
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))
            accuracy.append(acc/len(testloader))
        
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.6f}.. ".format(train_losses[e]),
                  "Test Loss: {:.6f}.. ".format(test_losses[e]),
                  "Test Accuracy: {:.6f}".format(accuracy[e])
                  )
            
        
    return -1





In [0]:
 train_model_1()

Epoch: 1/10..  Training Loss: 2.295185..  Test Loss: 2.275266..  Test Accuracy: 0.251919
Epoch: 2/10..  Training Loss: 2.201518..  Test Loss: 1.952294..  Test Accuracy: 0.651590
Epoch: 3/10..  Training Loss: 1.529691..  Test Loss: 0.764752..  Test Accuracy: 0.827303
Epoch: 4/10..  Training Loss: 0.940943..  Test Loss: 0.493012..  Test Accuracy: 0.860746
Epoch: 5/10..  Training Loss: 0.744956..  Test Loss: 0.395128..  Test Accuracy: 0.885691
Epoch: 6/10..  Training Loss: 0.650084..  Test Loss: 0.332801..  Test Accuracy: 0.894189
Epoch: 7/10..  Training Loss: 0.584911..  Test Loss: 0.295193..  Test Accuracy: 0.912007
Epoch: 8/10..  Training Loss: 0.529476..  Test Loss: 0.262136..  Test Accuracy: 0.907621
Epoch: 9/10..  Training Loss: 0.505346..  Test Loss: 0.244295..  Test Accuracy: 0.924890
Epoch: 10/10..  Training Loss: 0.471746..  Test Loss: 0.228448..  Test Accuracy: 0.923246


-1

In [0]:
#check predicitions:

model.to('cpu')

images, labels = next(iter(testloader))
# Get the class probabilities
ps = torch.exp(model(images))
# Make sure the shape is appropriate, we should get 10 class probabilities for 64 examples
print(ps.shape)

top_p, top_class = ps.topk(1, dim=1)
# Look at the most likely classes for the first 10 examples
print(top_class[10:15,:])
print(top_class.shape)


torch.Size([64, 10])
tensor([[3],
        [4],
        [8],
        [5],
        [5]])
torch.Size([64, 1])


# Let's divide train data(60000) into "*number _of_teachers*" private datasets :

In [0]:
def split_train_teacher(ds):

    num_teachers = 80
    batch_size = 64
    
    max_ind = len (ds)
    print(len(ds))
    
    teacher_len = int(max_ind/num_teachers)

    teacher_loader = []
    
    start = 0
    stop = teacher_len           
    
       
    #print(f"start = {start}",f"train_size = {train_size}",f"test_size = {test_size}")

    indicies = torch.randperm(max_ind)
    

    for i in range(num_teachers):
      
        idx =[j for j in range(start,stop)]
        idx = indicies[start:stop]
        
        #print("Teacher indicies = ",start," | ", stop, " | ",{stop-start})
    
        teacher_loader.append(torch.utils.data.DataLoader( ds, batch_size=batch_size, sampler = SubsetRandomSampler(idx)))

        #print(f"teacher_loader = {len(teacher_loader[i])}")
    
        start = stop
        stop = stop+teacher_len
        
    return  teacher_loader


In [12]:
#loaders for teacher training process
batch_size = 64

teacher_train_loader = split_train_teacher(train_dataset)    #60000/80 = 750 samples per teacher

#500 samples  reused for every teacher validation
teacher_valid_loader = torch.utils.data.DataLoader( test_valid_ds, batch_size=batch_size, shuffle = True)   

print(f"teacher_train_loader = {len(teacher_train_loader)}",f"teacher_valid_loader = {len(teacher_valid_loader)}")

60000
teacher_train_loader = 80 teacher_valid_loader = 8


In [0]:
#loader for fed_labels production (9500) from test_dataset (10000)

batch_size = 64
split = 9500
idx =[j for j in range(len(test_dataset))]
train_id = [j for j in range(split)]
test_id = [j for j in range(split,len(test_dataset))]

valid_loader = torch.utils.data.DataLoader( test_dataset, batch_size=batch_size, sampler = torch.utils.data.SequentialSampler(train_id))
test_loader = torch.utils.data.DataLoader( test_dataset, batch_size=batch_size, sampler = torch.utils.data.SequentialSampler(test_id))


In [0]:
len(train_id)

9500

In [0]:
len(test_id)

500

In [0]:
print(len(valid_loader),len(test_loader))

149 8


# Train model on 10 teacher datadets 
#Predict 10 sets of true_labels[ 10 ] on valid dataset :

In [0]:
#reset_model_conv()

In [0]:
def evaluate_model_t(device,testloader_t,criterion):
    
    test_loss = 0
    acc = 0

    model.eval()


    with torch.no_grad():
                
         for inputs, labels in testloader_t:

            #inputs = inputs.view(inputs.shape[0], -1)
            inputs, labels = inputs.to(device), labels.to(device)

            log_ps = model.forward(inputs)

            #print(labels.shape,log_ps.shape)
            test_loss += criterion(log_ps, labels)

            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            
            equals = top_class == labels.view(*top_class.shape)
            acc += torch.mean(equals.type(torch.FloatTensor))

            #test_count +=1
            
    return test_loss, acc 
  

def print_test_accuracy(e,epochs,accuracy):
    print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Test Accuracy: {:.6f}".format(accuracy[e])
                 )
        


In [0]:
def train_model_t(trainloader_t,testloader_t):
    
    #device = 'cuda'
    epochs = 12
    #count = 0

    model.to(device)

    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters())
    
    #testloader = testloader
    
    train_losses = []
    test_losses = []
    accuracy =  []
    
    true_labels = []
    dc_valid = []
    
    for e in range(epochs):
        model.train()
        running_loss = 0
        train_count = 0

        for images, labels in trainloader_t:

            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            output = model.forward(images)

            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            #train_count+=1
            
        else:
            #model evaluation
            test_loss,acc = evaluate_model_t(device,testloader_t,criterion)

            #collect accuracy, train_losses and test_losses for each epoch
            train_losses.append(running_loss/len(trainloader_t))
            test_losses.append(test_loss/len(testloader_t))
            accuracy.append(acc/len(testloader_t))
        
            #print("Epoch: {}/{}.. ".format(e+1, epochs),
             #     "Training Loss: {:.6f}.. ".format(train_losses[e]),
             #    "Test Loss: {:.6f}.. ".format(test_losses[e]),
             #    "Test Accuracy: {:.6f}".format(accuracy[e])
             #    )
            print_test_accuracy(e,epochs,accuracy) 
            
    return -1



In [0]:
#train teacher_model and make predictions

device = 'cuda'
preds = []
true_labels = []
dc_valid = torch.Tensor(0).cpu()

for t in range(num_teachers):
    reset_model_conv(model)
    print(f"Teacher {t+1}")
    #instansiate next teacher model
    model = Conv()
    
    
    #train next teacher model
    train_model_t(teacher_train_loader[t],teacher_valid_loader[t])
    
    batch_preds = []
    batch_count = 0
    dc_t = torch.Tensor(0).cpu()
    
    for images, label in valid_loader:
        
        #images, labels = images.to(device), labels.to(device)
        images = images.to(device)
        true_labels.append(label)
        
        ps = torch.exp(model.forward(images))
        top_p, top_class = ps.topk(1, dim=1)
        
        #batch_preds.append(top_class.flatten())
        dc_batch = top_class.flatten().float().cpu()        
        dc_t = torch.cat((dc_t,dc_batch),0).view(-1)
        

    #preds.append(np.array(batch_preds))
    dc_valid = torch.cat((dc_valid,dc_t),0)

#train teachers and produce federated labels

In [0]:
def evaluate_model_t_labels(device,testloader_t,criterion):
    
    test_loss = 0
    acc = 0
    #v_count = 0
    
    #dc_t = torch.Tensor(0).cpu()
    
    model.eval()


    with torch.no_grad():
                
        for inputs, labels in testloader_t:

            #inputs = inputs.view(inputs.shape[0], -1)
            inputs, labels = inputs.to(device), labels.to(device)

            log_ps = model.forward(inputs)

            #print(labels.shape,log_ps.shape)
            test_loss += criterion(log_ps, labels)

            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            acc += torch.mean(equals.type(torch.FloatTensor))
          
    return test_loss, acc
  

def print_test_accuracy(e,epochs,accuracy):
    print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Test Accuracy: {:.6f}".format(accuracy[e])
                 )
        


In [0]:
def train_model_t_labels(trainloader_t,testloader_t,valid_loader):
    
    #device = 'cuda'
    epochs = 12
    #count = 0

    model.to(device)

    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters())
    
    train_losses = []
    test_losses = []
    accuracy =  []
    
    
    for e in range(epochs):
        model.train()
        running_loss = 0
        train_count = 0

        for images, labels in trainloader_t:

            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            output = model.forward(images)

            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            #train_count+=1
            
        else:
            #model evaluation
            test_loss,acc = evaluate_model_t_labels(device,testloader_t,criterion)

            #collect accuracy, train_losses and test_losses for each epoch
            train_losses.append(running_loss/len(trainloader_t))
            test_losses.append(test_loss/len(testloader_t))
            accuracy.append(acc/len(testloader_t))
            
            
        
            #print("Epoch: {}/{}.. ".format(e+1, epochs),
             #     "Training Loss: {:.6f}.. ".format(train_losses[e]),
             #    "Test Loss: {:.6f}.. ".format(test_losses[e]),
             #    "Test Accuracy: {:.6f}".format(accuracy[e])
             #    )
            print_test_accuracy(e,epochs,accuracy)
            
    #produce true_labels
    
    acc_teacher = 0
    v_count = 0
    labels_teacher = torch.Tensor(0).cpu()
    ds_labels = torch.FloatTensor(0).cpu()
    
    #preserve sequence of images,labels in valid_loader
    imgs = []
    labs = []
    
    model.eval()


    with torch.no_grad():
                
        for inputs, labels in valid_loader:
            #print("batch # ",v_count)
            
            #preserve the sequence of images and labels
            imgs.extend(inputs)
            labs.extend(labels.flatten().numpy())
            
            
            
            inputs, labels = inputs.to(device), labels.to(device)

            log_ps = model.forward(inputs)
                                                 #print(labels.shape,log_ps.shape)
            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            acc_teacher += torch.mean(equals.type(torch.FloatTensor))
            
            #collect original MNIST labels
            ds_labels = torch.cat((ds_labels,labels.float().cpu()),0).view(-1)
            #print("dataset_labels ",len(ds_labels))
            
            
            #Federated learning labels batch tensor
            labels_batch = top_class.flatten().float().cpu()
            #print("dc_batch.size ", dc_batch.size())
            labels_teacher = torch.cat((labels_teacher,labels_batch),0).view(-1)
            #print("labels_teacher.size ",labels_teacher.size())
          
            
            v_count +=1
          
    #return labels_teacher, acc_teacher, mnist_labels
    
    
            
    return labels_teacher, ds_labels, acc_teacher, imgs, labs



In [18]:
num_teachers = 80
device = 'cuda'
#true_labels = torch.Tensor(0).cpu()
true_labels = []
mnist_labels = []
accuracy_teacher = []

for t in range(num_teachers):         
    reset_model_conv(model)
    
    print(f"Teacher {t}")
    #instansiate next teacher model
    model = Conv()
    
    #train next teacher model
    #labels_teacher, ds_labels, acc_teacher = train_model_t_labels(teacher_train_loader[t],teacher_valid_loader[t])
    labels_teacher, ds_labels, acc_teacher, imgs, labs = train_model_t_labels(teacher_train_loader[t], teacher_valid_loader,valid_loader)
    
    #collect true_labels, mnist_labels for the teacher
    true_labels.append(labels_teacher.cpu())
    mnist_labels.append(ds_labels.cpu())
    accuracy_teacher.append(acc_teacher.cpu())
    
#save sequence of images for local_model training

#imgs_iter = itertools.chain(imgs)
#labs_iter = itertools.chain(labs)
    #print(f"Teacher {t} has been trained\n\n")
    
print(f"All {t+1} Teachers  has been trained\n\n")

Teacher 0
Epoch: 1/12..  Test Accuracy: 0.095102
Epoch: 2/12..  Test Accuracy: 0.144231
Epoch: 3/12..  Test Accuracy: 0.620042
Epoch: 4/12..  Test Accuracy: 0.643329
Epoch: 5/12..  Test Accuracy: 0.780649
Epoch: 6/12..  Test Accuracy: 0.801532
Epoch: 7/12..  Test Accuracy: 0.827524
Epoch: 8/12..  Test Accuracy: 0.844501
Epoch: 9/12..  Test Accuracy: 0.862981
Epoch: 10/12..  Test Accuracy: 0.842097
Epoch: 11/12..  Test Accuracy: 0.863582
Epoch: 12/12..  Test Accuracy: 0.876202
Teacher 1
Epoch: 1/12..  Test Accuracy: 0.223708
Epoch: 2/12..  Test Accuracy: 0.224309
Epoch: 3/12..  Test Accuracy: 0.470252
Epoch: 4/12..  Test Accuracy: 0.602464
Epoch: 5/12..  Test Accuracy: 0.697266
Epoch: 6/12..  Test Accuracy: 0.773287
Epoch: 7/12..  Test Accuracy: 0.787710
Epoch: 8/12..  Test Accuracy: 0.812350
Epoch: 9/12..  Test Accuracy: 0.819712
Epoch: 10/12..  Test Accuracy: 0.840595
Epoch: 11/12..  Test Accuracy: 0.855619
Epoch: 12/12..  Test Accuracy: 0.849008
Teacher 2
Epoch: 1/12..  Test Accuracy

In [22]:
print(len(labs),len(imgs))

9500 9500


In [0]:
np.transpose(labs[:100])

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9])

In [23]:
print(f"len(mnist_labels) {len(mnist_labels)}",f"len(true_labels) {len(true_labels)}")


len(mnist_labels) 80 len(true_labels) 80


In [0]:
mnist_labels[0][:100]

tensor([7., 2., 1., 0., 4., 1., 4., 9., 5., 9., 0., 6., 9., 0., 1., 5., 9., 7.,
        3., 4., 9., 6., 6., 5., 4., 0., 7., 4., 0., 1., 3., 1., 3., 4., 7., 2.,
        7., 1., 2., 1., 1., 7., 4., 2., 3., 5., 1., 2., 4., 4., 6., 3., 5., 5.,
        6., 0., 4., 1., 9., 5., 7., 8., 9., 3., 7., 4., 6., 4., 3., 0., 7., 0.,
        2., 9., 1., 7., 3., 2., 9., 7., 7., 6., 2., 7., 8., 4., 7., 3., 6., 1.,
        3., 6., 9., 3., 1., 4., 1., 7., 6., 9.])

In [0]:
mnist_label = np.array( [ item.int() for item in mnist_labels[0].flatten() ] )


In [25]:
len(mnist_label)

9500

In [0]:
mnist_label.shape
mnist_label[:100]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9], dtype=int32)

In [26]:
len(true_labels)

80

In [0]:
true_labels[12][:100]

tensor([7., 2., 1., 0., 4., 1., 4., 4., 5., 9., 0., 6., 9., 0., 1., 5., 9., 7.,
        3., 4., 9., 6., 6., 5., 4., 0., 7., 4., 0., 1., 3., 1., 3., 0., 7., 2.,
        7., 1., 2., 1., 1., 7., 4., 2., 3., 5., 3., 2., 4., 4., 6., 3., 5., 5.,
        6., 0., 4., 1., 9., 5., 7., 8., 4., 3., 7., 4., 0., 4., 3., 0., 7., 0.,
        2., 8., 1., 7., 3., 7., 9., 7., 7., 6., 2., 7., 8., 4., 7., 3., 6., 1.,
        3., 6., 4., 3., 1., 4., 3., 1., 6., 9.])

In [0]:
true_label = [item.int() for sublist in true_labels for item in sublist.flatten()]

In [0]:
len(true_label)

760000

In [0]:
#predictions from num_teachers on test_train (9500)
preds = np.array(true_label).reshape(num_teachers,-1)

In [29]:
print(mnist_label.shape,preds.shape)

(9500,) (80, 9500)


In [0]:
acc_teacher/num_teachers

tensor(1.7457)

#Differential Privacy:

In [0]:
!pip install syft


In [20]:
import syft as sy
from syft.frameworks.torch.differential_privacy import pate

W0831 03:06:55.901216 140253058959232 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0831 03:06:55.914040 140253058959232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
mnist_label[:]

array([7, 2, 1, ..., 9, 0, 1], dtype=int32)

In [0]:
preds[:,0]

array([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7], dtype=int32)

In [0]:
num_labels = 10
label_counts = np.bincount(preds[:,0], minlength=num_labels)
label_counts

array([ 0,  0,  0,  0,  0,  0,  0, 80,  0,  0])

In [0]:
new_label = np.argmax(label_counts)
new_label

7

In [0]:
#iterate through all images
num_labels = 10

new_labels = list()
for an_image in np.transpose(preds):

    label_counts = np.bincount(an_image, minlength=num_labels)

    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)
    
new_labels = np.array(new_labels)


In [31]:
new_labels.shape

(9500,)

In [0]:
print(new_labels[:100])

[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 6 7 2 7
 1 2 1 1 7 4 2 3 5 3 2 4 4 6 3 5 5 6 0 4 1 9 5 7 8 4 3 7 4 6 4 3 0 7 0 2 7
 1 7 3 7 9 7 7 6 2 7 8 4 7 3 6 1 3 6 4 3 1 4 1 7 6 9]


In [0]:
#create dp_labels
#iterate through 9500 images

num_labels = 10

dp_labels = list()
for item in np.transpose(preds):

    label_counts = np.bincount(item, minlength=num_labels)

    epsilon = 0.1     #131.5/3000 = 0.044 68.77/3000 = 0.023
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)

    dp_label = np.argmax(label_counts)
    
    dp_labels.append(new_label)
    
dp_labels = np.array(new_labels)


In [33]:
dp_labels.shape

(9500,)

In [0]:
mnist_label[0:100]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9], dtype=int32)

In [0]:
new_labels[0:100]

array([7, 2, 1, 0, 4, 1, 4, 9, 6, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 6, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 3, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 4, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 7, 1, 7, 3, 7, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 4, 3, 1, 4, 1, 7, 6, 9])

In [0]:
dp_labels[0:100]

array([7, 2, 1, 0, 4, 1, 4, 9, 6, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 6, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 3, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 4, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 7, 1, 7, 3, 7, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 4, 3, 1, 4, 1, 7, 6, 9])

In [0]:
def fed_accuracy(alt_labels, true_labels):
    x =np.array( [(alt_labels[i] == true_labels[i]) for i in new_labels]).astype(int)
    fed_accuracy = sum(x)/len(new_labels)
    print(fed_accuracy)
    return -1

In [0]:
# test_dataset(10000)
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=dp_labels, noise_eps=0.1, delta=1e-5) #,moments =20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 411.5129254649703
Data Dependent Epsilon: 93.89830871784949


In [0]:
fed_accuracy(dp_labels, mnist_label)

0.9151578947368421


-1

In [0]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=dp_labels, noise_eps=0.1, delta=1e-5) #,moments =20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 131.51292546497027
Data Dependent Epsilon: 68.7707820260897


In [0]:
#data_ind_eps = 0.044
fed_accuracy(dp_labels, mnist_label)
#0.7833

0.7833333333333333


-1

In [0]:
#data_ind_eps = 0.023
fed_accuracy(dp_labels, mnist_label)
#0.7833

0.7833333333333333


-1

#create new dp_dataset = images+dp_labels

In [0]:
import numpy as np

In [0]:
new_ds = zip(dp_labels, imgs)

result = list(new_ds)
print(result[0])

In [0]:
local_ds = zip(imgs, dp_labels)

local_result = list(local_ds)
print(local_result[0])

In [0]:
tensor_x = torch.stack([torch.Tensor(r) for r in result[0][1]])
tensor_x.shape

torch.Size([1, 28, 28])

In [0]:
print(result[0][1].squeeze().shape)

torch.Size([28, 28])


In [0]:
x=np.array([sublist[0] for sublist in result])
x[:64].transpose()

array([7, 2, 1, 0, 4, 1, 4, 9, 6, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 6, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 3, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 4, 3])

In [0]:
imgs_arr = [item.numpy() for item in imgs]

len(imgs_arr)

9500

In [0]:
trainloader = torch.utils.data.DataLoader(dataset = local_result,  batch_size=64,sampler = torch.utils.data.SequentialSampler(local_result))

In [0]:
nlb = []
for label,image in trainloader:
  nlb.append(label)
  

In [0]:
nlb[0]

tensor([7, 2, 1, 0, 4, 1, 4, 9, 6, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 6, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 3, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 4, 3])

In [0]:
dp_labels[:64]

array([7, 2, 1, 0, 4, 1, 4, 9, 6, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 6, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 3, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 4, 3])

In [0]:
np.array(nlb[:64]).transpose()

#Create remote workers:

In [0]:
hook = sy.TorchHook(torch)

In [0]:
bob = sy.VirtualWorker(hook, id="bob")#.add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice")#.add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker")#.add_worker(sy.local_worker)

#train local model on test dataset

In [0]:
#trainloader = valid_loader
testloader = test_loader

In [0]:
def tensor_encrypt(tens):
    return tens.fix_precision().share(alice,bob, crypto_provider = secure_worker, requires_grad = True)

In [0]:
#encrypted dataloader
def encrypt_trainloader():
    encrypted_target = []
    encrypted_images = []
    for i, (image, target) in enumerate(trainloader):
        x = tensor_encrypt(target)     #torch.from_numpy(target)
        y = tensor_encrypt(torch.stack([torch.FloatTensor(r) for r in image]))

        encrypted_target.append(x)
        encrypted_images.append(y)
        
    encrypted_trainloader = list(zip(encrypted_images,encrypted_target))    
    return encrypted_trainloader

In [0]:
encrypted_trainloader = encrypt_trainloader()

In [45]:
encrypted_trainloader[0]

((Wrapper)>AutogradTensor>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:20392186314 -> alice:77919678737]
 	-> [PointerTensor | me:39774904561 -> bob:6846002777]
 	*crypto provider: secure_worker*,
 (Wrapper)>AutogradTensor>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:73363614545 -> alice:81744867945]
 	-> [PointerTensor | me:12988787576 -> bob:68724884360]
 	*crypto provider: secure_worker*)

In [0]:
img, target = next(iter(testloader))
print(img.type(),target.type(),target)

torch.FloatTensor torch.LongTensor tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3])


In [0]:
def encrypt_testloader():
    encrypted_target = []
    encrypted_images = []
    for i, (image, target) in enumerate(testloader):

        x = tensor_encrypt(target)     #torch.from_numpy(target)
        
        y = tensor_encrypt(image)

        encrypted_target.append(x)
        encrypted_images.append(y)
        
    encrypted_testloader = list(zip(encrypted_images, encrypted_target))    
    return encrypted_testloader

In [0]:
encrypted_testloader = encrypt_testloader()

In [48]:
encrypted_testloader[0]

((Wrapper)>AutogradTensor>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:52203504153 -> alice:51235638974]
 	-> [PointerTensor | me:17220461085 -> bob:31109358671]
 	*crypto provider: secure_worker*,
 (Wrapper)>AutogradTensor>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:97521740341 -> alice:45049752039]
 	-> [PointerTensor | me:22886434918 -> bob:13814933053]
 	*crypto provider: secure_worker*)

In [49]:
images,labels =next(iter(trainloader))
print(images.shape, labels.shape)

torch.Size([64, 1, 28, 28]) torch.Size([64])


In [50]:
local_model = Conv()
local_model

Conv(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [51]:
local_model.fix_precision().share(alice,bob, crypto_provider = secure_worker, requires_grad = True)

Conv(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [0]:
reset_model_conv(local_model)

-1

In [0]:
def evaluate_model_1(device,criterion,optimizer):
    
    #device = device
    test_loss = 0
    acc = 0

    model.eval()


    with torch.no_grad():
                
         for inputs, labels  in encrypted_testloader:

            #inputs = inputs.view(inputs.shape[0], -1)
            inputs, labels = inputs.to(device), labels.to(device)

            log_ps = model.forward(inputs)
            
            #print(labels.shape,log_ps.shape)
            test_loss += criterion(log_ps, labels).item()

            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            
            #print(f"top_p: {top_p}")
            equals = top_class == labels.view(*top_class.shape)
            #print(f"equals: {equals}")
            acc += torch.mean(equals.type(torch.FloatTensor))
            #print(f"acc: {acc}")
            #test_count +=1
            
    return test_loss, acc      


In [0]:
def train_model_1():
    
    device = 'cuda'
    epochs = 1
    count = 0

    #model.to(device)

    criterion = nn.NLLLoss()
    #optimizer = optim.Adam(model.parameters(),lr = 0.01)
    optimizer = optim.SGD(model.parameters(),lr= 0.01, momentum = 0.5)
    optimizer.fix_precision()
    
    train_losses = []
    test_losses = []
    accuracy =  []
    
    for e in range(epochs):
        model.train()
        running_loss = 0
        train_count = 0

        for labels, images in encrypted_trainloader:

            images, labels = images.to(device), labels.to(device)

            # TODO: Training pass
            optimizer.zero_grad()

            #output = model(images)
            output = model.forward(images)

            #print(output.shape, labels.shape)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            #print(train_count)
            train_count+=1
            
        else:
            #model evaluation
            test_loss,acc = evaluate_model_1(device,criterion,optimizer)

            #collect accuracy, train_losses and test_losses for each epoch
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))
            accuracy.append(acc/len(testloader))
        
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.6f}.. ".format(train_losses[e]),
                  "Test Loss: {:.6f}.. ".format(test_losses[e]),
                  "Test Accuracy: {:.6f}".format(accuracy[e])
                  )
          
          
    return -1



In [0]:
train_count = 0
for images, labels in trainloader:
    
    if train_count == int(len(test_train_ds)/batch_size):
        print(f"Num_samples = {train_count*batch_size}")
        break
    train_count+=1  
    print(train_count)

In [54]:
 #train local_model on test_ds
  train_model_1()
  
 

RuntimeError: ignored

In [0]:
#Epoch: 15/15..  Training Loss: 0.282330..  Test Loss: 0.357445..  Test Accuracy: 0.900691
#Epoch: 15/15..  Training Loss: 0.272720..  Test Loss: 0.347765..  Test Accuracy: 0.904147